#Name: Joseph Ereifej
##ECE2975 Reinforcement Learning 2023 - Homework Assignment 1: MDPs and Bellman Equations

Total: 100 points

Topics covered on this Assignment:

Markov Decission Processes (MDPs)

Probabilities of transition in T steps
Example: "Student MDP"
Dinamic programming:

Policy Improvement Theorem
Policy Evaluation
Policy Iteration
Value Iteration
Completion of this assignment entails answering questions and writing code sections.

Suggested lecture: Sutton & Barto 2020 Caps. 3 y 4 (link)


In [1]:
# Libraries to load
import numpy as np
from matplotlib import pyplot as plt

# 1. T-step state probabilities (25pt)


## 1.1


\begin{equation}
P(S_T = s^\prime | S_0 = s)=\\
\end{equation}
if T == 1:
\begin{equation}
\sum P(S_T = s^\prime | A_0 = A_i) \pi (A_i | S_0)
\end{equation}
else
\begin{equation}
\sum P(S_T = s^\prime | S_{T-1} = S_i, A_{T-1} = A_j) \pi (A_j | S_i)*P(S_{T-1} = s_i | S_0 = s)
\end{equation}




Briefly explain your response:
    This is a recursive solution for the generalized probability of being in a final state S' given and initial state S0

## 1.2

$$P(\mathcal T_3=\texttt{happy,books,happy,books,sad,party,sad})=0.004*P(S_0)$$

Briefly explain your response:
![](ECE2795HW1%20Pr%201.2.jpg)

### 1.3 Setup

In [18]:
# The following classes implement the MDP and a deterministic policy


class StudentMDP(object):
  """
Basic class implementing the MDP.
  Use get_data() to obtain transition probabilities and rewards.
  """

  ACTION_PARTY = 0
  ACTION_STUDY = 1
  STATE_HAPPY = 0
  STATE_SAD = 1

  def __init__(self):
      self.data = [[[[], []], [[], []]], [[[], []], [[], []]]]
      self.data[StudentMDP.STATE_HAPPY][StudentMDP.ACTION_PARTY][StudentMDP.STATE_HAPPY] = (1. , 10.)
      self.data[StudentMDP.STATE_HAPPY][StudentMDP.ACTION_PARTY][StudentMDP.STATE_SAD] = (0., 0.)
      self.data[StudentMDP.STATE_HAPPY][StudentMDP.ACTION_STUDY][StudentMDP.STATE_HAPPY] = (0.2, -10.)
      self.data[StudentMDP.STATE_HAPPY][StudentMDP.ACTION_STUDY][StudentMDP.STATE_SAD] = (0.8, -10.)
      self.data[StudentMDP.STATE_SAD][StudentMDP.ACTION_PARTY][StudentMDP.STATE_HAPPY] = (0.8, 40.)
      self.data[StudentMDP.STATE_SAD][StudentMDP.ACTION_PARTY][StudentMDP.STATE_SAD] = (0.2, 40)
      self.data[StudentMDP.STATE_SAD][StudentMDP.ACTION_STUDY][StudentMDP.STATE_HAPPY] = (0.8, 20)
      self.data[StudentMDP.STATE_SAD][StudentMDP.ACTION_STUDY][StudentMDP.STATE_SAD] = (0.2, 20)

  def get_data(self, state, action, state_prime):
    # Computes the transition probabillities and the reward going from 'state'
    # to 'state_prime' under 'action'.
    # Eg.: p, r = get_data(StudentMDP.STATE_HAPPY, StudentMDP.ACTION_PARTY, StudentMDP.STATE_SAD)

      if (state < 0 or state >= self.get_num_states()):
          raise Exception("Invalid state")
      if (state_prime < 0 or state_prime >= self.get_num_states()):
          raise Exception("Invalid state_prime")
      if (action < 0 or action >= self.get_num_actions()):
          raise Exception("Invalid action")
      return self.data[state][action][state_prime]

  def get_num_states(self):
    # Number of states
      return 2

  def get_num_actions(self):
    # Number of actions
      return 2


class DeterministicPolicy(object):
  """
  Class describing a deterministic policy. Example:

  policy = DeterministicPolicy(2, 2)
  for state in [StudentMDP.STATE_HAPPY, StudentMDP.STATE_SAD]:
    policy.set_action(state, StudentMDP.ACTION_PARTY)
  """

  def __init__(self, num_states, num_actions):
      self.num_states = num_states
      self.num_actions = num_actions
      self.state_action_map = np.zeros(num_states, dtype=np.int32)

  def get_action(self, state):
      if (state < 0 or state >= self.num_states):
          raise Exception("Invalid state")
      return self.state_action_map[state]

  def set_action(self, state, action):
      if (state < 0 or state >= self.num_states):
          raise Exception("Invalid state")
      if (action < 0 or action >= self.num_actions):
          raise Exception("Invalid action")
      self.state_action_map[state] = action

### 1.3.1
Write a function to compute the probability of being at   $s'$ after  $T$ steps, starting  from $s$, and following a deterministic policy.


In [42]:
student = StudentMDP()
num_states = student.get_num_states()
num_actions = student.get_num_states()

policy = DeterministicPolicy(num_states, num_actions)
for state in [StudentMDP.STATE_HAPPY, StudentMDP.STATE_SAD]:
    policy.set_action(state, StudentMDP.ACTION_PARTY)


def probMDP(state, state_prime, T_steps):
    if T_steps == 1:
        prob_sum = 0
        for action in range(num_actions):
            # deterministic probability
            policy_prob = 0
            if action == 0:
                policy_prob = 1
            prob_sum = prob_sum + student.get_data(state, action, state_prime)[0]*policy_prob
        return prob_sum
    else:
        prob_sum = 0
        for statei in range(num_states):
            for actionj in range(num_actions):
                policy_prob = 0
                if actionj == 0:
                    policy_prob = 1
                prob_sum = prob_sum + student.get_data(statei, actionj, state_prime)[0]*policy_prob*probMDP(state, statei, T_steps-1)
        return prob_sum



### 1.3.2

Using this code compute


1.   $P(S_1 =\texttt{happy} \mid S_0 = \texttt{happy})=1.0$
2.   $P(S_1 = \texttt{happy} \mid S_0 = \texttt{sad})=0.8$
3.   $P(S_2 = \texttt{happy} \mid S_0 = \texttt{happy})=1.0$
4.   $P(S_2 = \texttt{happy} \mid S_0 = \texttt{sad})=0.96$
5.   $P(S_3 = \texttt{happy} \mid S_0 = \texttt{happy})=1.0$
6.   $P(S_3 = \texttt{happy} \mid S_0 = \texttt{sad})=0.992$

In [43]:
# ------------
print(probMDP(StudentMDP.STATE_HAPPY, StudentMDP.STATE_HAPPY, 1))
print(probMDP(StudentMDP.STATE_SAD, StudentMDP.STATE_HAPPY, 1))
print(probMDP(StudentMDP.STATE_HAPPY, StudentMDP.STATE_HAPPY, 2))
print(probMDP(StudentMDP.STATE_SAD, StudentMDP.STATE_HAPPY, 2))
print(probMDP(StudentMDP.STATE_HAPPY, StudentMDP.STATE_HAPPY, 3))
print(probMDP(StudentMDP.STATE_SAD, StudentMDP.STATE_HAPPY, 3))
# ------------

1.0
0.8
1.0
0.9600000000000001
1.0
0.9920000000000001


Compute $P(S_2 = \texttt{happy} | S_0 = \texttt{sad})$ using  (1.1) and compare with the code output.

**Your answer:**
![](ECE2795HW1%20Pr%201.3.jpg)

# Discussion:
Working out each time step allowed me to better understand the algorithm for computing the T-Step probabilities

---
# 2. Policy Improvement

Proof below





## 2.1

Prove $$\mathbb{E}_{a \sim \pi^\prime}[q_\pi(s, a)] \geq v_\pi(s).$$



## 2.2

Prove $$Q_2 := \mathbb{E}_{\pi^\prime}[R_{t+1} + \gamma R_{t+2} + \gamma^2 v_\pi(S_{t+2})] \geq Q_1.$$



## 2.3
Prove
$$Q_{T+1} \geq Q_{T}.$$




## 2.4
Conclude the proof by arguing that $Q_\infty \geq v_\pi(s)$.



Brief discussion
---
![](ECE2795HW1%20Pr%202.jpg)

# 3. Policy Iteration


## 3.1




In [205]:
import matplotlib.pyplot as plt
# Complete this function.
# Implements Policy Evaluation. The inputs are an object of class StudentMDP
# and a policy to be evaluated. Returns a numpy array of size (num_states,)
# with the value function for each state
def PolicyEvaluation(mdp, policy, gamma=.9):
    N = mdp.get_num_states()
    value = np.zeros(N)

    theta = 0.1

    idx = 0
    while True:
        delta = 0
        V_new = 0
        for state in range(N):
            action = policy.get_action(state)
            for next_state in range(N):
                p, r = mdp.get_data(state, action, next_state)
                V_new = V_new + p*(r + gamma*value[next_state])
            pass
            delta = max(delta, np.abs(V_new - value[state]))
            value[state] = V_new
        if delta < theta:
            break

        idx = idx + 1
    return value

# Complete this function.
# Implements Policy Improvement. The inputs are an MDP and a value function
# (obtained by applying PolicyEvaluation()), computes the q-function for each state
# and returns a new policy, greedy with respect to q.

def PolicyImprovement(mdp, value_func, gamma=.9):

    N = mdp.get_num_states()
    M = mdp.get_num_actions()
    policy = DeterministicPolicy(N, M)
    for state in [StudentMDP.STATE_HAPPY, StudentMDP.STATE_SAD]:
        policy.set_action(state, StudentMDP.ACTION_STUDY)

    for state in range(N):
        new_policy = policy
        for action in range(M):
            new_policy.set_action(state, np.argmax(PolicyEvaluation(mdp, policy)))
        pass
        # set policy to new policy
        policy = new_policy

    return policy


# Complete this function
# The input is two policies  old_policy and new_policy,
# being both of the class DeterministicPolicy().
# Returns True or False indicating if PolicyIteration should stop or not.
def ShouldTerminate(mdp, old_policy, new_policy):
    if (old_policy.state_action_map == new_policy.state_action_map).all():
        return True
    return False



def PolicyIteration(mdp, initial_policy, gamma=.9):
    policy = initial_policy
    value_data = []
    idx = 0
    while True:
        value_func = PolicyEvaluation(mdp, policy, gamma)
        value_data.append(value_func)
        new_policy = PolicyImprovement(mdp, value_func, gamma)

        should_terminate = ShouldTerminate(mdp, policy, new_policy)
        if should_terminate:
            break
        policy = new_policy
        idx += 1
    return policy

## 3.2

Apply this code for the student model of problem 1. Provide the following graphs:



1.   Value function $v(s)$  for each state ($\texttt{happy}$, $\texttt{sad})$ as a function of  the excecition time $k$, being $k$ the number of runs of  Policy Evaluation.
![](pr3.png)
2.   Table with $\pi(a|s)$ upon convergence.

| state | action |
| ------------- | ------------- |
| Happy  | Study  |
| Sad  | Party  |
(for GitHub)

3. Table with $v(s)$ upon convergence.

| state | value |
| ------------- | ------------- |
| Happy  | -7.55  |
| Sad  | 32.5  |
(for GitHub)


In [206]:
student = StudentMDP()
num_states = student.get_num_states()
num_actions = student.get_num_states()

policy = DeterministicPolicy(num_states, num_actions)
for state in [StudentMDP.STATE_HAPPY, StudentMDP.STATE_SAD]:
    policy.set_action(state, StudentMDP.ACTION_STUDY)

policy_Pol_Iter = PolicyIteration(student, policy, gamma=.1)

C:\Users\ereij\AppData\Local\Temp\ipykernel_22984\2473680647.py:20: RuntimeWarning: overflow encountered in scalar add
  V_new = V_new + p*(r + gamma*value[next_state])
C:\Users\ereij\AppData\Local\Temp\ipykernel_22984\2473680647.py:22: RuntimeWarning: invalid value encountered in scalar subtract
  delta = max(delta, np.abs(V_new - value[state]))


From a computational point of view, can you think of a problem with this method?

Discuss briefly:
You need to compute the evaluation for each policy every iteration. If it doesn't converge quickly then you may be left waiting a long time until the solution converges.

# 4. Value iteration

## 4.1
Implement the Value Iteration algorithm




In [207]:
def ValueIteration(mdp, initial_policy, gamma=.9):
    N = mdp.get_num_states()
    M = mdp.get_num_actions()

    value = np.zeros(N)

    theta = 0.1
    # Value Iteration
    while True:
        delta = 0
        V_new = 0
        for state in range(N):
            for action in range(M):
                V_action = 0
                for next_state in range(N):
                    p, r = mdp.get_data(state, action, next_state)
                    V_action = V_action + p*(r + gamma*value[next_state])
                pass
                V_new = max(V_new, V_action)
            delta = max(delta, np.abs(V_new - value[state]))
            value[state] = V_new
        if delta < theta:
            break

    # Make the Policy
    policy = initial_policy
    for state in range(N):
        Action_Eval = np.zeros(M)
        for action in range(M):
            for next_state in range(N):
                p, r = mdp.get_data(state, action, next_state)
                Action_Eval[action] = Action_Eval[action] + p*(r + gamma*value[next_state])
        policy.set_action(state, np.argmax(Action_Eval))
    return policy


## 4.2

 Use your code for the MDP in problem 1. Provide the following graphs:


1.   Value function $v(s)$  for each state ($\texttt{happy}$, $\texttt{sad})$ as a function of  the excecition time $k$, being $k$ the number of times the outer loop is run.
![](pr3.png)
2.   Table with $\pi(a|s)$ upon convergence.

| state | action |
| ------------- | ------------- |
| Happy  | Study  |
| Sad  | Party  |
(for GitHub)

3. Table with $v(s)$ upon convergence.

| state | value |
| ------------- | ------------- |
| Happy  | -7.55  |
| Sad  | 32.5  |
(for GitHub)

In [208]:
student = StudentMDP()
num_states = student.get_num_states()
num_actions = student.get_num_states()

policy = DeterministicPolicy(num_states, num_actions)
for state in [StudentMDP.STATE_HAPPY, StudentMDP.STATE_SAD]:
    policy.set_action(state, StudentMDP.ACTION_STUDY)

policy_Val_Iter = PolicyIteration(student, policy, gamma=.1)

C:\Users\ereij\AppData\Local\Temp\ipykernel_22984\2473680647.py:20: RuntimeWarning: overflow encountered in scalar add
  V_new = V_new + p*(r + gamma*value[next_state])
C:\Users\ereij\AppData\Local\Temp\ipykernel_22984\2473680647.py:22: RuntimeWarning: invalid value encountered in scalar subtract
  delta = max(delta, np.abs(V_new - value[state]))


Discuss briefly, comparing with PI in part 3.

The policy is equivalent to that of part 3 except the runtime is significantly shorter

## 5 Bellman equations


### 5.1 Write the Bellman equations for the student model

In [209]:
def BellmanEquation(mdp, policy, gamma=.9):
    N = mdp.get_num_states()
    M = mdp.get_num_actions()

    value = np.zeros(N)
    for state in range(N):
        action = policy.get_action(state)
        for next_state in range(N):
            p, r = mdp.get_data(state, action, next_state)
            value[state] = value[state] + p*(r + gamma*value[next_state])

    return value

student = StudentMDP()
num_states = student.get_num_states()
num_actions = student.get_num_states()

policy = DeterministicPolicy(num_states, num_actions)
for state in [StudentMDP.STATE_HAPPY, StudentMDP.STATE_SAD]:
    policy.set_action(state, StudentMDP.ACTION_STUDY)

print(BellmanEquation(student, policy))

[-10.     14.384]



### 5.2 Write a piece of code to compute the Bellman error for this MDP

In [210]:
student = StudentMDP()
num_states = student.get_num_states()
num_actions = student.get_num_states()

# Bellman Equation before and after runnning another Policy Iteration
print(BellmanEquation(student, policy_Pol_Iter))
print(BellmanEquation(student, PolicyIteration(student, policy_Pol_Iter)))
print("Error: " + str(BellmanEquation(student, policy_Pol_Iter) - BellmanEquation(student, PolicyIteration(student, policy_Pol_Iter))))

# Bellman Equation before and after runnning another Value Iteration
print(BellmanEquation(student, policy_Val_Iter))
print(BellmanEquation(student, ValueIteration(student, policy_Val_Iter)))
print("Error: " + str(BellmanEquation(student, policy_Val_Iter) - BellmanEquation(student, ValueIteration(student, policy_Val_Iter))))

[-10.     37.264]
[-10.     37.264]
Error: [0. 0.]
[-10.     37.264]
[-10.     37.264]
Error: [0. 0.]


C:\Users\ereij\AppData\Local\Temp\ipykernel_22984\2473680647.py:20: RuntimeWarning: overflow encountered in scalar add
  V_new = V_new + p*(r + gamma*value[next_state])
C:\Users\ereij\AppData\Local\Temp\ipykernel_22984\2473680647.py:22: RuntimeWarning: invalid value encountered in scalar subtract
  delta = max(delta, np.abs(V_new - value[state]))


# 6 Concluding remarks:
This HW helped solidify my knowledge of MDP, value functions, and policies. It also helped me better understand the algorithms behind fundamental reinforcement learning problems. I think the most difficult part of the assignment was understanding the math syntax of the algorithms and equations, as well as working through the proofs, but after solving them I have gained a better understanding for then.